# Transforming Problems

## The `BasisTransformer`

In [44]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis

driver = PySCFDriver()
driver.run_pyscf()

ao_problem = driver.to_problem(basis=ElectronicBasis.AO)
print(ao_problem.basis)

ao_hamil = ao_problem.hamiltonian
print(ao_hamil.electronic_integrals.alpha)

ElectronicBasis.AO
Polynomial Tensor
 "+-":
[[-1.12421758 -0.9652574 ]
 [-0.9652574  -1.12421758]]
 "++--":
[[[[0.77460594 0.44744572]
   [0.44744572 0.3009177 ]]

  [[0.44744572 0.3009177 ]
   [0.57187698 0.44744572]]]


 [[[0.44744572 0.57187698]
   [0.3009177  0.44744572]]

  [[0.3009177  0.44744572]
   [0.44744572 0.77460594]]]]


Next, we obtain the `MO` to `AO` transformer:

In [45]:
from qiskit_nature.second_q.formats.qcschema_translator import get_ao_to_mo_from_qcschema

qcschema = driver.to_qcschema()

basis_transformer = get_ao_to_mo_from_qcschema(qcschema)
print(basis_transformer.initial_basis)
print(basis_transformer.final_basis)

ElectronicBasis.AO
ElectronicBasis.MO


And finally, we can use the transformer to obtain the problem in the `MO` basis:

In [46]:
mo_problem = basis_transformer.transform(ao_problem)
print(mo_problem.basis)

mo_hamil = mo_problem.hamiltonian
print(mo_hamil.electronic_integrals.alpha)

ElectronicBasis.MO
Polynomial Tensor
 "+-":
[[-1.25633907e+00 -6.21867875e-17]
 [-7.78036432e-17 -4.71896007e-01]]
 "++--":
[[[[6.75710155e-01 1.12401641e-16]
   [1.56722377e-16 1.80931200e-01]]

  [[1.92605510e-16 1.80931200e-01]
   [6.64581730e-01 2.59298923e-16]]]


 [[[8.68926823e-17 6.64581730e-01]
   [1.80931200e-01 1.82411770e-16]]

  [[1.80931200e-01 2.57172666e-16]
   [7.20426423e-17 6.98573723e-01]]]]


If you need to construct your `BasisTransformer` manually, you can do this like so:

In [47]:
import numpy as np
from qiskit_nature.second_q.operators import ElectronicIntegrals
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import BasisTransformer

ao2mo_alpha = np.random.random((2, 2))
ao2mo_beta = np.random.random((2, 2))

basis_transformer = BasisTransformer(
    ElectronicBasis.AO,
    ElectronicBasis.MO,
    ElectronicIntegrals.from_raw_integrals(ao2mo_alpha, h1_b=ao2mo_beta),
)

## The `FreezeCoreTransformer`

In [48]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(atom="Be 0 0 0; H 0 0 -1.32; H 0 0 1.32")

full_problem = driver.run()
print(full_problem.molecule)
print(full_problem.num_particles)
print(full_problem.num_spatial_orbitals)

Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Bohr
	Geometry:
		Be	(0.0, 0.0, 0.0)
		H	(0.0, 0.0, -2.494438484425882)
		H	(0.0, 0.0, 2.494438484425882)
	Masses:
		Be	9
		H	1
		H	1
(3, 3)
7


In [49]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

fc_transformer = FreezeCoreTransformer()

fc_problem = fc_transformer.transform(full_problem)
print(fc_problem.num_particles)
print(fc_problem.num_spatial_orbitals)

(2, 2)
6


In [50]:
print(fc_problem.hamiltonian.constants)

{'nuclear_repulsion_energy': 3.407580524863637, 'FreezeCoreTransformer': -15.042985348309728}


In [51]:
fc_transformer = FreezeCoreTransformer(remove_orbitals=[6, 7])

fc_problem = fc_transformer.transform(full_problem)
print(fc_problem.num_particles)
print(fc_problem.num_spatial_orbitals)

(2, 2)
5


## The `ActiveSpaceTransformer`

In [52]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(atom="Be 0 0 0; H 0 0 -1.32; H 0 0 1.32")

full_problem = driver.run()
print(full_problem.num_particles)
print(full_problem.num_spatial_orbitals)

(3, 3)
7


In [53]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

as_transformer = ActiveSpaceTransformer(4, 4)

as_problem = as_transformer.transform(full_problem)
print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)
print(as_problem.hamiltonian.electronic_integrals.alpha)

(2, 2)
4
Polynomial Tensor
 "+-":
[[-1.52068450e+00 -2.21108986e-16  7.17364791e-17 -1.07170108e-16]
 [-5.68135777e-17 -1.52143297e+00  2.40565490e-16  2.53569377e-16]
 [ 7.17364791e-17  2.40565490e-16 -1.17917888e+00 -1.71735667e-16]
 [-1.07170108e-16  2.53569377e-16 -6.41748405e-17 -1.17917888e+00]]
 "++--":
[[[[ 3.99710434e-01  2.96196380e-15 -2.02831538e-17  3.02406615e-17]
   [ 2.93544929e-15  1.64638880e-01 -2.02645725e-17 -1.78914898e-17]
   [-2.02831538e-17 -2.02645725e-17  4.95573429e-02 -1.83699563e-18]
   [ 3.02406615e-17 -1.78914898e-17  8.26109358e-18  4.95573429e-02]]

  [[ 2.99520573e-15  1.64638880e-01 -2.02645725e-17 -1.78914898e-17]
   [ 4.13083362e-01 -2.59049574e-15 -2.11198447e-17  3.17279887e-17]
   [-7.92245968e-17 -1.04748907e-17 -3.03735111e-16  5.34481717e-19]
   [-7.71489254e-17  1.62388599e-17  5.73808690e-19 -3.02878681e-16]]

  [[-2.02831538e-17 -2.02645725e-17  4.95573429e-02  6.06381353e-18]
   [-7.92245968e-17 -1.04748907e-17 -3.03485447e-16 -2.44367009

In [54]:
as_transformer = ActiveSpaceTransformer(2, 2, active_orbitals=[0, 4])

as_problem = as_transformer.transform(full_problem)
print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)
print(as_problem.hamiltonian.electronic_integrals.alpha)

(1, 1)
2
Polynomial Tensor
 "+-":
[[-6.79068552e+00  2.03675789e-18]
 [ 2.03675789e-18 -9.10997635e-01]]
 "++--":
[[[[ 2.27148263e+00 -2.10555106e-18]
   [-2.10555106e-18  1.57674385e-02]]

  [[-2.10555106e-18  1.57674385e-02]
   [ 5.69173075e-01 -3.31781299e-20]]]


 [[[-2.10555106e-18  5.69173075e-01]
   [ 1.57674385e-02 -3.31781299e-20]]

  [[ 1.57674385e-02 -3.31781299e-20]
   [-3.31781299e-20  4.49859041e-01]]]]
